# Implementação do PKE-IND-CCA (NTRU-PRIME)

In [1]:
def randomprime(i):
    return random_prime(2**i-1,True,2**(i-1))

In [2]:
def verifyW(p, q, w, indice):
    while (2*p < 3*w):
        indice = indice-1
        w = randomprime(indice)
    
    while (q < (16*w + 1)):
        indice = indice-1
        w = randomprime(indice)
    
    return w

- p
- q
- w

In [3]:
l = 8

p = randomprime(l)
q = randomprime(l)
w = randomprime(l)

print p
print q
print w

w = verifyW(p,q,w,l)

print w

RingQ.<x> = PolynomialRing(GF(q),'x')
(x^p-x-1).is_irreducible()

Z = IntegerRing()
Ring  = PolynomialRing(Z,'x')
Ring3 = PolynomialRing(GF(3),'x')

R  = QuotientRing(Ring, Ring.ideal(x^p-x-1))
R3 = QuotientRing(Ring3, Ring3.ideal(x^p-x-1))
Rq = QuotientRing(RingQ, RingQ.ideal(x^p-x-1))

199
193
227
11


In [4]:
def roundEach(f,b=None):
    ff = list(f)
    if b == None:
        return ff
    else:
        fp = map(lift,[Mod(a,b) for a in ff])
        return [u if u <= b//2 else u-b for u in fp ]

def toR(vec):
    return R(vec)

def _h(f,g):
    _f = Rq(roundEach(f)) ; _g = Rq(roundEach(g))
    try:
        fq = _f.inverse_of_unit()
        hq = fq * _g
        return (True,toR([lift(a) for a in list(hq)]))
    except:
        return (False,0)

In [5]:
import random as rn

def small(P=p):
    u = [rn.choice([-1,0,1]) for i in range(p)]
    return u

def smallW(P=p,W=w):
    u = [rn.choice([-1,1]) for i in range(w)] + [0]*(p-w)
    rn.shuffle(u)
    return u

In [6]:
def KeyGen(j=q, k=p, l=w):
    while True:
        G = small()
        F = smallW()
        f = 1 + 3 * toR(F)
        g = 3 * toR(G)
        (flag, h) = _h(f,g)
        if flag:
            break
    return {'sk' : f , 'pk' : h} 

def Encapsulate(plaintext,pk):
    r = toR(smallW())
    m = toR(plaintext)
    return roundEach(pk*r + m, b=q)

def Decapsulate(ciphertext,sk):
    e = toR(ciphertext)
    a = roundEach(sk*e, b=q)
    return roundEach(toR(a), b=3)
    
def run():
    keys   = KeyGen()
    plain  = small()
    crypto = Encapsulate(plain,keys['pk'])
    decryp = Decapsulate(crypto,keys['sk'])
    return plain == decryp

In [7]:
run()

True